In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px


In [2]:
from features.titanic_feature_builder import TitanicFeatureBuilder

In [3]:
titanic = TitanicFeatureBuilder(pd.read_csv('Titanic.csv'))

In [4]:
titanic.df.head()

In [5]:
titanic.df.info()

In [6]:
titanic.df['Age'].head()

## Sex to male

In [7]:
titanic.sex_to_male()
titanic.df.head()

## Embarked encode

In [8]:
print(titanic.df['Embarked'].unique())
print(titanic.df['Embarked'].value_counts())
titanic.fill_embarked()
print(titanic.df['Embarked'].unique())
print(titanic.df['Embarked'].value_counts())

In [9]:
# ендокінг в hot-endoding без dummy-trap
titanic.encode_embarked()
titanic.df.head()

# Add missing (без embarked)

In [10]:
titanic.add_missing()
titanic.df.isnull().sum()

# Name to status

In [11]:
# Бачимо що всі значення мають певний патерн, де можна побачити статус між , та .
# Одна запись з графинею не відповідає цьому патерну, для неї застосуємо додатковий патеррн
titanic.df['Name'].value_counts().head()

In [12]:
titanic.add_status_by_name()
titanic.df.Status.value_counts()
# Спробуємо поділити статуси по тутулу/ролі на такі групи:
# діти, цивільні, еліта(титульні), special (екіпаж, або військові. Доктори не входять в категорію, бо: 
# 1. Не має інформації доктори наук чи звичайні 
# 2. За інформацією, під час удару не було травм для людей та під час затоплення допомога докторів 
# не така значуща, тому вони діють як цивільні)

In [13]:
titanic.add_role_by_status()
survival_rates = titanic.df.groupby("Role").agg(Survived=('Survived', 'mean')).reset_index()
fig = px.histogram(survival_rates, x="Role", y="Survived", histnorm="probability")
fig.show()
# Розподіл показує, що діти та еліта мали більше шанс вижити

In [14]:
titanic.encode_role()
titanic.df.head()

# Fill age by median

In [15]:
titanic.df.groupby(['Status']).aggregate(mean_age=('Age', 'median'))

In [16]:
titanic.fill_age()
titanic.df.head()

In [17]:
titanic.df['Age'].isnull().sum()

In [18]:
titanic.df['Age'].hist()

# Cabin

In [19]:
print(titanic.df['Cabin'].value_counts())
print(titanic.df['Cabin'].isnull().sum())

## Каюту можна спробувати заповнити за номером класу, для цього спочатку продивимость всі значення кают

In [20]:
titanic.df.info()

In [21]:
titanic.split_cabins()
titanic.df.head()

In [22]:
','.join(titanic.df.query('Pclass == 1').loc[:, 'Cabin'].dropna().drop_duplicates().astype(str))

In [23]:
','.join(titanic.df.query('Pclass == 2').loc[:, 'Cabin'].dropna().drop_duplicates().astype(str))

In [24]:
','.join(titanic.df.query('Pclass == 3').loc[:, 'Cabin'].dropna().drop_duplicates().astype(str))

In [25]:
fig = px.histogram(titanic.df, x="Pclass", color="Deck")
fig.show()

In [26]:
# Бачимо, що: 
# E є в усіх класах, 
# D є в 1 та 2 класі
# F є в 2 та 3 класі

# A, B, C, T тільки в 1 класу,     оскільки кілька варіантів, заповніти не вийде
# G тільки в 3 класу, тому спробуємо заповнювати,            малий приріст інформації

# Можна спробувати знайти закономірність між ціною та палублю

In [27]:
fig = px.histogram(titanic.df, x="Fare", color="Deck", nbins=10)
fig.show()

# Як результат, бачимо, що немає чіткої закономірності між ціною квитка та палубою, отже позначимо всі невідомі палуби як unknown
# Палуба T має лише 1 елемент, тому спочатку подивимость на цей елемент

In [28]:
# Чогось особливого в елементу немає, оскільки елемент 1 з такою палубою кинемо каюту в unknown
titanic.df.query('Deck == "T"')

In [29]:
titanic.fill_decks()
titanic.df['Deck'].value_counts()

In [30]:
titanic.encode_decks()
titanic.df.head()

# Age та Fare to bins

In [31]:
fig = px.histogram(titanic.df, x="Age", color="Survived", nbins=20)
fig.show()

In [32]:
titanic.add_bins_age(drop_old=True)
titanic.df.head()

# Нормалізація Age та Fare

In [33]:
fig = px.histogram(titanic.df, x="Fare", color="Survived", nbins=10)
fig.show()

In [34]:
titanic.add_bins_fare(drop_old=True)
titanic.df.head()

# Для алгоритму k-means важлива нормалізація

In [35]:
titanic.normalize_age()
titanic.df['Age'].describe()

In [36]:
titanic.normalize_fare()
titanic.df['Fare'].describe()

# Що можна зробити ще

In [37]:
# PessengerId, Name, Ticker мають високу ентропію, можна викинути
# Cabin має багату кількість важко відновлених даних
# Room має глибокий зв'язок з Deck, кімната більш поточнує локацію людини, але через занадто багату
# кількість важковідновлених даних скіпну фічу

# з SibSp та Parch можна вижати корисну інформацію про знаходження родин поряд

titanic.df.head()

# FamilySize (аріфметика)

In [38]:
titanic.add_family_size()
titanic.df.head()

In [39]:
fig = px.histogram(titanic.df, x="SibSp", color="Survived")
fig.show()

In [40]:
survival_rates = titanic.df.groupby("SibSp").agg(Survived=('Survived', 'mean')).reset_index()
fig = px.bar(survival_rates, x="SibSp", y="Survived")
fig.show()

# Створимо фічі Alone, Small family(1-2) та Big family(3+)

In [41]:
titanic.split_family_size()
titanic.df.head()

## SibSp

In [42]:
titanic.df['SibSp'].describe()

In [43]:
fig = px.histogram(titanic.df, x="SibSp", color="Survived")
fig.show()
# На гістограми бачимо, що від біна 4 до 8 надто висока смертність, але в цій області
# надто мало даних
# досліджу більш детально розподіл від 1 до 8

In [44]:
survival_rates = titanic.df.groupby("SibSp").agg(Survived=('Survived', 'mean')).reset_index()
fig = px.bar(survival_rates, x="SibSp", y="Survived")
fig.show()
# Розподіл досить нерівномірний, тому зроблю хот енкодінг на: 0, 1, 2, 3, 4 та все що більше 4 буде в 4

In [45]:
titanic.encode_sibSp()
titanic.df.head()

## Parch

In [46]:
# Такий же принцип

In [47]:
titanic.df['Parch'].describe()

In [48]:
fig = px.histogram(titanic.df, x="Parch", color="Survived")
fig.show()

In [49]:
survival_rates = titanic.df.groupby("Parch").agg(Survived=('Survived', 'mean')).reset_index()
fig = px.bar(survival_rates, x="Parch", y="Survived")
fig.show()
# в людей з дітьми досить великий шанс вижити, можливо це реклама демографії від творця датасету, але
# сумніваюсь
# в бінах 3 - 6 досить мало значень та важко сказати як точно вони заповнювались, бо бін 6 має 
# n записів 1, бін 5 має 5, а бін 4 має 4 записи. Скоріше за все, бін 6 - помилка запису
# Тому в цей раз через досить велику різницю даних між тими хто має дітей та тими, хто не має
# Я зроблю фічу child-parent

In [50]:
titanic.parch_to_child_parent()
titanic.df.head()

In [51]:
titanic.df['Child_Parent'].value_counts()

# Кореляція

In [52]:
titanic.df.corr(numeric_only=True) 
# Cabin_missing досить корельована з Pclass(0.725) та з Deck_unknown(0.966), викину її
# Fare_bins досить корельован з Pclass(0.69), Fare(0.62) та Deck_unknown(0.72), викину Pclass та Fare
# Age_bins корелює з Age, викину Age
# SibSp_1 корелює з FamilySize_Small(0.68), викину SibSp_1
# Також викину де-які інфі високоентропійні х-ки

In [53]:
titanic.drop_irrelevant()
titanic.df.head()

In [54]:
testtitanic = TitanicFeatureBuilder(pd.read_csv('Titanic_test.csv'))
testtitanic.build_by_template().head()

In [55]:
testtitanic.df.corr(numeric_only=True) 
